In [19]:
import pandas as pd
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from sklearn.preprocessing import StandardScaler

In [20]:
file_path = "/Users/amulya/Desktop/Capstone/DSCI-601-Amy/Data/FeatureEngineered/AKAM_feature_engineeredv2.csv"  
data = pd.read_csv(file_path)

data.fillna(method='ffill', inplace=True)

/var/folders/69/00tjyfg95_d37pp6ql47yf9h0000gn/T/ipykernel_22703/3782561420.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


In [21]:
data.dropna(inplace=True)

In [22]:
data.isnull().sum()

date               0
TICKER             0
CUSIP              0
COMNAM             0
RET                0
VOL_CHANGE         0
BA_SPREAD          0
ILLIQUIDITY        0
sprtrn             0
TURNOVER           0
PRC                0
SHROUT             0
MARKET_CAP         0
TRAN_COST          0
ASK                0
BID                0
day_of_week        0
day_of_month       0
month              0
is_month_start     0
is_month_end       0
year               0
week               0
RET_ema_12         0
RET_ema_26         0
RET_skew_10        0
RET_kurtosis_10    0
RET_entropy_10     0
RSI                0
OBV                0
RET_lag_1          0
RET_lag_7          0
RET_lag_14         0
RET_lag_30         0
RET_mean_7d        0
RET_std_7d         0
RET_mean_14d       0
RET_std_14d        0
RET_mean_30d       0
RET_std_30d        0
dtype: int64

In [23]:

features = data[['RET', 'VOL_CHANGE', 'BA_SPREAD', 'ILLIQUIDITY', 'TURNOVER', 
                 'PRC', 'RET_ema_12', 'RET_ema_26', 'RET_mean_7d', 'RET_std_7d',
                 'RET_mean_14d', 'RET_std_14d', 'RET_mean_30d', 'RET_std_30d',
                 'RET_lag_7', 'RET_lag_14', 'RET_lag_30']]


features.dropna(inplace=True)


scaler = StandardScaler()
features_scaled = pd.DataFrame(scaler.fit_transform(features), columns=features.columns)

/var/folders/69/00tjyfg95_d37pp6ql47yf9h0000gn/T/ipykernel_22703/3464024915.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.dropna(inplace=True)


In [24]:
# Split data into train and test sets
train_size = int(len(features_scaled) * 0.8)
train_data = features_scaled.iloc[:train_size]
test_data = features_scaled.iloc[train_size:]


## Dealing with error

In [27]:
correlation_matrix = train_data.corr()
print(correlation_matrix)


                   RET  VOL_CHANGE  BA_SPREAD  ILLIQUIDITY  TURNOVER  \
RET           1.000000    0.075179  -0.006117     0.319487  0.055670   
VOL_CHANGE    0.075179    1.000000   0.016066     0.035431  0.453740   
BA_SPREAD    -0.006117    0.016066   1.000000    -0.090281 -0.109243   
ILLIQUIDITY   0.319487    0.035431  -0.090281     1.000000  0.015381   
TURNOVER      0.055670    0.453740  -0.109243     0.015381  1.000000   
PRC           0.005423   -0.004495  -0.253646     0.014881 -0.158624   
RET_ema_12    0.532967    0.016284  -0.058537     0.141220  0.040748   
RET_ema_26    0.376231    0.014461  -0.086502     0.094526  0.032103   
RET_mean_7d   0.024772    0.018645  -0.052983     0.024939  0.022818   
RET_std_7d    0.015955   -0.009560   0.311321    -0.014535  0.095379   
RET_mean_14d  0.013294    0.007688  -0.057732     0.002154  0.008829   
RET_std_14d  -0.005605    0.010422   0.354830    -0.027782  0.059623   
RET_mean_30d  0.002021   -0.004415  -0.091419     0.007240 -0.02

In [28]:
train_data_diff = train_data.diff().dropna()


In [29]:
from statsmodels.tsa.stattools import adfuller

for column in train_data.columns:
    result = adfuller(train_data[column])
    print(f"{column}: p-value = {result[1]}")


RET: p-value = 1.5723486085406141e-21
VOL_CHANGE: p-value = 0.0
BA_SPREAD: p-value = 0.09944660110140957
ILLIQUIDITY: p-value = 6.1764167394065375e-24
TURNOVER: p-value = 3.8724411282419324e-10
PRC: p-value = 5.731132621498917e-10
RET_ema_12: p-value = 4.6764006838330663e-20
RET_ema_26: p-value = 3.381943041704588e-17
RET_mean_7d: p-value = 2.923347457564208e-20
RET_std_7d: p-value = 1.4252451942593623e-05
RET_mean_14d: p-value = 4.59135195942962e-18
RET_std_14d: p-value = 8.897501856669607e-05
RET_mean_30d: p-value = 6.147707251335246e-14
RET_std_30d: p-value = 0.007363946714567
RET_lag_7: p-value = 1.4307768132633018e-21
RET_lag_14: p-value = 1.238451990536758e-21
RET_lag_30: p-value = 8.479202464632231e-22


In [30]:
from sklearn.covariance import LedoitWolf

cov_estimator = LedoitWolf()
train_data_regularized = pd.DataFrame(cov_estimator.fit_transform(train_data), columns=train_data.columns)


AttributeError: 'LedoitWolf' object has no attribute 'fit_transform'

In [25]:
from statsmodels.tsa.api import VAR

# Fit the VAR model
model = VAR(train_data)
results = model.fit(maxlags=15, ic='aic')  # Use AIC for optimal lag selection

# Print the results summary
print(results.summary())


LinAlgError: 16-th leading minor of the array is not positive definite

In [ ]:
# Forecast for the test data
forecast_steps = len(test_data)
forecast = results.forecast(train_data.values[-results.k_ar:], steps=forecast_steps)

# Convert forecast to DataFrame
forecast_df = pd.DataFrame(forecast, columns=features_scaled.columns, index=test_data.index)

# Rescale forecast back to original scale
forecast_original_scale = pd.DataFrame(scaler.inverse_transform(forecast_df), 
                                       columns=features.columns, 
                                       index=test_data.index)


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Evaluate the forecast for each feature
for column in features.columns:
    actual = features[column].iloc[train_size:].values  # Actual values
    predicted = forecast_original_scale[column].values  # Predicted values
    
    mae = mean_absolute_error(actual, predicted)
    mse = mean_squared_error(actual, predicted)
    r2 = r2_score(actual, predicted)
    
    print(f"Feature: {column}")
    print(f"  MAE: {mae:.4f}")
    print(f"  MSE: {mse:.4f}")
    print(f"  R²: {r2:.4f}")


In [ ]:
import matplotlib.pyplot as plt

# Plot actual vs predicted values for key features
for column in ['RET', 'PRC']:  # Replace with the features you're most interested in
    plt.figure(figsize=(10, 6))
    plt.plot(features[column].iloc[train_size:].values, label='Actual', color='blue')
    plt.plot(forecast_original_scale[column].values, label='Forecast', color='orange', linestyle='--')
    plt.title(f"{column} - Actual vs Forecast")
    plt.legend()
    plt.show()
